In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import cross_val_predict, cross_val_score, StratifiedKFold

import pandas as pd
import numpy as np

## Carrega base

In [3]:
# Com categorização das colunas bmi e glicose
stroke_cat = pd.read_csv('stroke_df_cat')


In [4]:
stroke_cat.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,28,0,0,1,2,1,1,1,2,0
1,1,33,0,0,1,2,0,1,4,1,0
2,0,42,0,0,1,2,0,2,3,0,0
3,1,56,0,0,1,2,1,1,5,2,0
4,0,24,0,0,0,2,0,1,5,2,0


## Entrada para modelagem

In [5]:
#features

X = stroke_cat.drop(['stroke', 'Residence_type', 'ever_married'], axis=1).values

#target

y = (stroke_cat['stroke']).values

## Criação dos modelos

In [6]:
ad_clf = DecisionTreeClassifier()
knn_clf = KNeighborsClassifier(n_neighbors = 5)
rf_clf = RandomForestClassifier(n_estimators=100)
lg_clf = LogisticRegression(random_state=0, max_iter=1000)
mlp_clf = MLPClassifier(random_state=1, max_iter=300)

## Amostragem Holdout teste dos modelos

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33)

In [8]:
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X, y, test_size = 0.33,stratify=y)

#stratify, a função train_test_split garante que a divisão dos dados em conjuntos de treinamento e teste mantém a mesma distribuição das classes em y nos dois conjuntos

## Oversampling

In [12]:
from imblearn.over_sampling import SMOTE

s=SMOTE()
X_reso,y_reso =s.fit_resample(X_train,y_train)


## Undersampling

In [13]:
from imblearn.under_sampling import ClusterCentroids,RandomUnderSampler

rus= RandomUnderSampler()
cc = ClusterCentroids()

X_resu,y_resu = rus.fit_resample(X_train,y_train)


## Treinar modelos - com Holdout

In [14]:
modelos = [ad_clf, knn_clf, rf_clf, lg_clf, mlp_clf]

# Dicionário para armazenar métricas
metrics = {"Modelo": [], "Accuracy": [], "Precision": [], "Recall": [], "F1-Score": []}

for modelo in modelos:
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    
    modelo_name = modelo.__class__.__name__
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    metrics["Modelo"].append(modelo_name)
    metrics["Accuracy"].append(accuracy)
    metrics["Precision"].append(precision)
    metrics["Recall"].append(recall)
    metrics["F1-Score"].append(f1)

# Exiba as métricas

metrics_df = pd.DataFrame(metrics)
print(metrics_df)


                   Modelo  Accuracy  Precision    Recall  F1-Score
0  DecisionTreeClassifier  0.935248   0.188776  0.180488  0.184539
1    KNeighborsClassifier  0.956436   0.307692  0.058537  0.098361
2  RandomForestClassifier  0.949505   0.209302  0.087805  0.123711
3      LogisticRegression  0.959802   1.000000  0.009756  0.019324
4           MLPClassifier  0.959406   0.000000  0.000000  0.000000


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Treinar modelos - com Holdout stratify

In [15]:
modelos = [ad_clf, knn_clf, rf_clf, lg_clf, mlp_clf]

# Dicionário para armazenar métricas
metrics = {"Modelo": [], "Accuracy": [], "Precision": [], "Recall": [], "F1-Score": []}

for modelo in modelos:
    modelo.fit(X_train_s, y_train_s)
    y_pred = modelo.predict(X_test_s)
    
    modelo_name = modelo.__class__.__name__
    accuracy = accuracy_score(y_test_s, y_pred)
    precision = precision_score(y_test_s, y_pred)
    recall = recall_score(y_test_s, y_pred)
    f1 = f1_score(y_test_s, y_pred)
    
    metrics["Modelo"].append(modelo_name)
    metrics["Accuracy"].append(accuracy)
    metrics["Precision"].append(precision)
    metrics["Recall"].append(recall)
    metrics["F1-Score"].append(f1)

# Exiba as métricas

metrics_df = pd.DataFrame(metrics)
print(metrics_df)

                   Modelo  Accuracy  Precision    Recall  F1-Score
0  DecisionTreeClassifier  0.940594   0.224242  0.177033  0.197861
1    KNeighborsClassifier  0.956634   0.375000  0.071770  0.120482
2  RandomForestClassifier  0.950099   0.273684  0.124402  0.171053
3      LogisticRegression  0.958614   0.500000  0.004785  0.009479
4           MLPClassifier  0.956634   0.375000  0.071770  0.120482


## Treinar modelos com  o KFold

In [16]:
modelos = [ad_clf, knn_clf, rf_clf, lg_clf, mlp_clf]


metrics = {"Modelo": [], "Accuracy": [], "Precision": [], "Recall": [], "F1-Score": []}

# Realize a validação cruzada e calcule as métricas para cada modelo
for modelo in modelos:
    
    modelo_name = modelo.__class__.__name__ 
    skf = StratifiedKFold(n_splits=5, shuffle=True)   
   
    y_pred = cross_val_predict(modelo, X, y, cv=skf)
    
    # Calcule as métricas
    
    accuracy = cross_val_score(modelo, X, y, cv=skf, scoring='accuracy').mean()
    precision = cross_val_score(modelo, X, y, cv=skf, scoring='precision').mean()
    recall = cross_val_score(modelo, X, y, cv=skf, scoring='recall').mean()
    f1 = cross_val_score(modelo, X, y, cv=skf, scoring='f1').mean()
    
    # Armazene as métricas no dicionário
    metrics["Modelo"].append(modelo_name)
    metrics["Accuracy"].append(accuracy)
    metrics["Precision"].append(precision)
    metrics["Recall"].append(recall)
    metrics["F1-Score"].append(f1)

# Exiba as métricas
import pandas as pd
metrics_df = pd.DataFrame(metrics)
print(metrics_df)


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\User\AppData\Local\Pro

                   Modelo  Accuracy  Precision    Recall  F1-Score
0  DecisionTreeClassifier  0.940992   0.193913  0.166142  0.171558
1    KNeighborsClassifier  0.955434   0.387365  0.087002  0.126476
2  RandomForestClassifier  0.951447   0.254181  0.120185  0.137551
3      LogisticRegression  0.958505   0.220000  0.004737  0.018509
4           MLPClassifier  0.958440   0.090909  0.014198  0.038175


## Treinar modelos com  Holdout e undersampling

In [17]:
modelos = [ad_clf, knn_clf, rf_clf, lg_clf, mlp_clf]

# Dicionário para armazenar métricas
metrics = {"Modelo": [], "Accuracy": [], "Precision": [], "Recall": [], "F1-Score": []}

for modelo in modelos:
    modelo.fit(X_resu, y_resu)
    y_pred = modelo.predict(X_test)
    
    modelo_name = modelo.__class__.__name__
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    metrics["Modelo"].append(modelo_name)
    metrics["Accuracy"].append(accuracy)
    metrics["Precision"].append(precision)
    metrics["Recall"].append(recall)
    metrics["F1-Score"].append(f1)

# Exiba as métricas

metrics_df = pd.DataFrame(metrics)
print(metrics_df)

                   Modelo  Accuracy  Precision    Recall  F1-Score
0  DecisionTreeClassifier  0.764554   0.119195  0.751220  0.205745
1    KNeighborsClassifier  0.791287   0.129258  0.721951  0.219259
2  RandomForestClassifier  0.781188   0.129325  0.765854  0.221283
3      LogisticRegression  0.785941   0.138017  0.814634  0.236042
4           MLPClassifier  0.717822   0.113924  0.878049  0.201681


## Treinar modelos com  Holdout e oversampling

In [18]:
modelos = [ad_clf, knn_clf, rf_clf, lg_clf, mlp_clf]

# Dicionário para armazenar métricas
metrics = {"Modelo": [], "Accuracy": [], "Precision": [], "Recall": [], "F1-Score": []}

for modelo in modelos:
    modelo.fit(X_reso, y_reso)
    y_pred = modelo.predict(X_test)
    
    modelo_name = modelo.__class__.__name__
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    metrics["Modelo"].append(modelo_name)
    metrics["Accuracy"].append(accuracy)
    metrics["Precision"].append(precision)
    metrics["Recall"].append(recall)
    metrics["F1-Score"].append(f1)

# Exiba as métricas

metrics_df = pd.DataFrame(metrics)
print(metrics_df)

                   Modelo  Accuracy  Precision    Recall  F1-Score
0  DecisionTreeClassifier  0.868317   0.114094  0.331707  0.169788
1    KNeighborsClassifier  0.888911   0.153696  0.385366  0.219750
2  RandomForestClassifier  0.877426   0.126354  0.341463  0.184453
3      LogisticRegression  0.791881   0.134083  0.756098  0.227774
4           MLPClassifier  0.806139   0.126448  0.639024  0.211120


## Treinar modelos com  Kfold e undersampling

In [19]:
modelos = [ad_clf, knn_clf, rf_clf, lg_clf, mlp_clf]
metrics = {"Modelo": [], "Accuracy": [], "Precision": [], "Recall": [], "F1-Score": []}

n_splits = 5
cv = StratifiedKFold(n_splits=n_splits,shuffle=True)

under = RandomUnderSampler()


for modelo in modelos:

    accuracies = []
    precisions = []
    recalls = []
    f1s = []    

    # Realize a validação cruzada com undersampling
    for train_idx, test_idx in cv.split(X, y):
        X_train, y_train = X[train_idx], y[train_idx]
        X_test, y_test = X[test_idx], y[test_idx]
        
    # Realize o undersampling nos dados de treinamento
        X_resampled, y_resampled =under.fit_resample(X_train, y_train)
    
        modelo_name = modelo.__class__.__name__  
        modelo.fit(X_resampled, y_resampled)
        y_pred = modelo.predict(X_test)
    
        modelo_name = modelo.__class__.__name__
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)
        
    mean_accuracy = sum(accuracies) / n_splits
    mean_precision = sum(precisions) / n_splits
    mean_recall = sum(recalls) / n_splits
    mean_f1 = sum(f1s) / n_splits

    # Armazene as métricas no dicionário
    metrics["Modelo"].append(modelo_name)
    metrics["Accuracy"].append(mean_accuracy)    
    metrics["Precision"].append(mean_precision)
    metrics["Recall"].append(mean_recall)
    metrics["F1-Score"].append(mean_f1)

# Exiba as métricas

metrics_df = pd.DataFrame(metrics)
print(metrics_df)



                   Modelo  Accuracy  Precision    Recall  F1-Score
0  DecisionTreeClassifier  0.738808   0.100396  0.669304  0.174537
1    KNeighborsClassifier  0.788799   0.133235  0.746844  0.226078
2  RandomForestClassifier  0.759719   0.123424  0.788114  0.213359
3      LogisticRegression  0.778671   0.135403  0.808549  0.231928
4           MLPClassifier  0.707704   0.111206  0.860780  0.196768


## Treinar modelos com  Kfold e oversampling

In [20]:
modelos = [ad_clf, knn_clf, rf_clf, lg_clf, mlp_clf]
metrics = {"Modelo": [], "Accuracy": [], "Precision": [], "Recall": [], "F1-Score": []}

n_splits = 5
cv = StratifiedKFold(n_splits=n_splits)
over = SMOTE()


for modelo in modelos:

    accuracies = []
    precisions = []
    recalls = []
    f1s = []    

    # Realize a validação cruzada com undersampling
    for train_idx, test_idx in cv.split(X, y):
        X_train, y_train = X[train_idx], y[train_idx]
        X_test, y_test = X[test_idx], y[test_idx]

    # Realize o undersampling nos dados de treinamento
        X_resampled, y_resampled =over.fit_resample(X_train, y_train)
    
        modelo_name = modelo.__class__.__name__  
        modelo.fit(X_resampled, y_resampled)
        y_pred = modelo.predict(X_test)
    
        modelo_name = modelo.__class__.__name__
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)
        
    mean_accuracy = sum(accuracies) / n_splits
    mean_precision = sum(precisions) / n_splits
    mean_recall = sum(recalls) / n_splits
    mean_f1 = sum(f1s) / n_splits

    # Armazene as métricas no dicionário
    metrics["Modelo"].append(modelo_name)
    metrics["Accuracy"].append(mean_accuracy)    
    metrics["Precision"].append(mean_precision)
    metrics["Recall"].append(mean_recall)
    metrics["F1-Score"].append(mean_f1)

# Exiba as métricas

metrics_df = pd.DataFrame(metrics)
print(metrics_df)

                   Modelo  Accuracy  Precision    Recall  F1-Score
0  DecisionTreeClassifier  0.861661   0.117861  0.360792  0.177491
1    KNeighborsClassifier  0.895837   0.171384  0.397150  0.239319
2  RandomForestClassifier  0.869307   0.121800  0.348194  0.180317
3      LogisticRegression  0.788668   0.130383  0.726347  0.221057
4           MLPClassifier  0.803893   0.130212  0.655218  0.216595


In [ ]:
pipelines = {
    'Decision Tree': Pipeline([
        ('classifier', DecisionTreeClassifier())
    ]),
    'Random Forest': Pipeline([
        ('classifier', RandomForestClassifier())
    ]),
    'MLP Classifier': Pipeline([
        ('classifier', MLPClassifier())
    ]),
    'KNN': Pipeline([
        ('classifier', KNeighborsClassifier())
    ]),
}

param_grids = {
    'Random Forest':{
        'classifier__n_estimators': [50,100,200],
        'classifier__max_depth': [2,4,6]
    },
    'Decision Tree':{
        'classifier__min_samples_split': [2,3,4],
        'classifier__max_depth': [2,4,6]
    },
    'KNN':{
        'classifier__n_neighbors': [3,5,7],
        'classifier__weights': ['uniform', 'distance']
    },
    'MLP Classifier':{
        'classifier__hidden_layer_sizes': [(50,), (100,50), (100,100,50)],
        'classifier__alpha': [0.0001, 0.001, 0.01]
    }
}

for model_name, pipeline in pipelines.items():
  grid_search = GridSearchCV(pipeline, param_grids[model_name], scoring='accuracy')
  grid_search.fit(X_train, y_train)

  best_params = grid_search.best_params_
  best_model = grid_search.best_estimator_

  y_pred = best_model.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)

  print("Modelo:", model_name)
  print("Melhores Parâmetros:", best_params)
  print("Accuracy:", accuracy)